In [1]:
%matplotlib inline
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# Loading your IBM Q account(s)
provider = IBMQ.load_account()

/opt/conda/lib/python3.8/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


In [2]:
#parameters
qubits_num=10 
PREPROCESS=True
w=32 #image width
h=32 #image height
batch_size=4 #???

In [3]:
import torchvision
import torch
from torchvision import transforms
import numpy as np

def load_data():
    train=torchvision.datasets.CIFAR10(root='./data', train=True, download=True,
                          transform=transforms.Compose([torchvision.transforms.Grayscale(num_output_channels=1),
                                                       transforms.ToTensor()]))
    train_loader=torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    test=torchvision.datasets.CIFAR10(root='./data', train=False, download=True,
                        transform=transforms.Compose([torchvision.transforms.Grayscale(num_output_channels=1),
                                                      transforms.ToTensor()]))
    test_loader = torch.utils.data.DataLoader(test, batch_size=1, shuffle=True)
    classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    return train,test



In [4]:
def getY1Y2(train,test):
    Y1,Y2=[],[]
    car_ind,truck_ind=train.class_to_idx['cat'],train.class_to_idx['dog']
    for i in range(len(train)):
        cclass=train[i][1]
        if cclass==car_ind:
            Y1.append(1)        
        else:
            Y1.append(-1)
    for i in range(len(train)):
        cclass=train[i][1]
        if cclass==truck_ind:
            Y2.append(1)
        else:
            Y2.append(-1)        
    Y1=np.asarray(Y1)
    Y2=np.asarray(Y2)
    return Y1,Y2 

In [8]:
import pennylane as qml
from pennylane.templates.embeddings import AmplitudeEmbedding
#from pennylane import numpy as np
from PIL import Image

dev=qml.device("default.qubit",wires=10)
@qml.qnode(dev)
def convert_pixels_to_qubits(image):
    #convert to array of shape "(2**n)"
    #print("before",image)
    im=image[0].squeeze().tolist()   
    #im=np.array(im)
    im=np.reshape(im,32*32*1)
    #print("after",im)
    #using pennylane Amplitude Embedding
    AmplitudeEmbedding(im,wires=range(10),pad=0.,normalize=True)
    return qml.expval(qml.PauliZ(wires=0))

In [9]:
#Pre-processing the dataset
if PREPROCESS==True:
    train,test=load_data()
    qtrain=[]
    print("Start Quantum pre-processing of the train dataset")
    for i, item in enumerate(train):
        print("{}/{}        ".format(i + 1, len(train)), end="\r")
        convert_pixels_to_qubits(item)
        qtrain.append(dev.state)
    qtrain=np.asarray(qtrain)
    print("Finish Quantum pre-prcessing of the train dataset")
    
    qtest=[]
    print("Start Quantum pre-prcessing of the test dataset")
    for i, item in enumerate(test):
        print("{}/{}        ".format(i + 1, len(test)), end="\r")
        convert_pixels_to_qubits(item)
        #qtest.append(dev.state)
    qtest=np.asarray(qtest)
    print("Finish Quantum pre-prcessing of the test dataset")

Files already downloaded and verified
Files already downloaded and verified
Start Quantum pre-processing of the train dataset
Finish Quantum pre-prcessing of the train dataset
Start Quantum pre-prcessing of the test dataset
Finish Quantum pre-prcessing of the test dataset


In [50]:
#Pre-processing the dataset
if PREPROCESS==True:
    train,test=load_data()
    qtrain=[]
    print("Start Quantum pre-processing of the train dataset")
    for i, item in enumerate(train):
        print("{}/{}        ".format(i + 1, len(train)), end="\r")
        convert_pixels_to_qubits(item)
        qtrain.append(dev.state)
    qtrain=np.asarray(qtrain)
    print("Finish Quantum pre-prcessing of the train dataset")
    
    qtest=[]
    print("Start Quantum pre-prcessing of the test dataset")
    for i, item in enumerate(test):
        print("{}/{}        ".format(i + 1, len(test)), end="\r")
        convert_pixels_to_qubits(item)
        #qtest.append(dev.state)
    qtest=np.asarray(qtest)
    print("Finish Quantum pre-prcessing of the test dataset")

Files already downloaded and verified
Files already downloaded and verified
Start Quantum pre-processing of the train dataset
before [tensor([[[0.23921569, 0.17647059, 0.1882353 , ..., 0.53725493,
           0.50980395, 0.50588238],
          [0.07450981, 0.        , 0.03921569, ..., 0.37254903,
           0.35294119, 0.36862746],
          [0.09411765, 0.03529412, 0.12156863, ..., 0.35294119,
           0.35686275, 0.3137255 ],
          ...,
          [0.67843139, 0.60392159, 0.6156863 , ..., 0.52549022,
           0.14117648, 0.14901961],
          [0.57254905, 0.50588238, 0.56470591, ..., 0.60000002,
           0.27058825, 0.23529412],
          [0.59215689, 0.53725493, 0.57647061, ..., 0.74117649,
           0.48627451, 0.3882353 ]]], dtype=float32, requires_grad=True)
 tensor(6, requires_grad=True)]
after [0.23921569 0.17647059 0.1882353  ... 0.74117649 0.48627451 0.3882353 ]


/opt/conda/lib/python3.8/site-packages/autograd/numpy/numpy_wrapper.py:77: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return _np.array(args, *array_args, **array_kwargs)


Traceback (most recent call last):
  File "<ipython-input-50-a7efb0bd2c65>", line 8, in <module>
    convert_pixels_to_qubits(item)
  File "/opt/conda/lib/python3.8/site-packages/pennylane/tape/qnode.py", line 530, in __call__
    self.construct(args, kwargs)
  File "/opt/conda/lib/python3.8/site-packages/pennylane/tape/qnode.py", line 477, in construct
    raise qml.QuantumFunctionError(
QuantumFunctionError: A quantum function must return either a single measurement, or a nonempty sequence of measurements.

Use %tb to get the full traceback.


In [10]:
#check
print('qtrain',qtrain)

qtrain [[0.01608674+0.j 0.01186727+0.j 0.01265842+0.j ... 0.04984251+0.j
  0.03270091+0.j 0.02610798+0.j]
 [0.03724749+0.j 0.0291881 +0.j 0.02243562+0.j ... 0.02984156+0.j
  0.03005938+0.j 0.02984156+0.j]
 [0.05054429+0.j 0.05014787+0.j 0.05014787+0.j ... 0.01645167+0.j
  0.01645167+0.j 0.01664989+0.j]
 ...
 [0.03566274+0.j 0.03591388+0.j 0.03591388+0.j ... 0.01180386+0.j
  0.00979469+0.j 0.00678094+0.j]
 [0.03681706+0.j 0.03628605+0.j 0.03610904+0.j ... 0.03292295+0.j
  0.03345397+0.j 0.03345397+0.j]
 [0.05009625+0.j 0.05183872+0.j 0.05162091+0.j ... 0.03855233+0.j
  0.0357208 +0.j 0.03550299+0.j]]


In [11]:
Y1,Y2=getY1Y2(train,test)